# *Téléchargement de documents issus de Gallica : téléchargement des textes bruts.*


Script de récupération automatique des fichiers .txt des documents océrisés par la BNF. 


Entrée: fichier .xls contenant la liste des liens ark des fichiers océrisés par la BNF à télécharger

Sortie: fichers .txt avec le contenu de l'océrisation (texte)

In [ ]:
#@markdown # Connexion à son compte Google Drive:

#@markdown Lancer la cellule, cliquer sur le lien généré en dessous qui ouvrira un nouvel onglet du navigateur. 

#@markdown Cliquer sur son compte google, accepter les autorisations, copier le lien qui s'afficher et le coller dans la case prévu à cet effet en dessous de cette cellule.

from google.colab import drive
from openpyxl import load_workbook
import pandas as pd
import re
import requests
import shutil
import os
import xml.etree.ElementTree as ET
import lxml
import glob
from lxml import etree
from re import match
from bs4 import BeautifulSoup
from urllib.error import HTTPError
import urllib.request, urllib.error, urllib.parse
import string
drive.mount('/content/drive/')
%cd /content/drive/My Drive/

In [ ]:
#@markdown # Téléchargement à partir d'un fichier excel contenant les liens vers les fichiers Gallica.

def txt_gallica_xls(chemin_vers_le_fichier_excel):
  wb= load_workbook(chemin_vers_le_fichier_excel)
  ws= wb[nom_de_la_feuille]
  column_id = ws[lettre_de_la_colonne]
  id_list = [column_id[x].value for x in range(len(column_id))]
  for e in id_list:
    if e == None:
      exit()
    else:
      match = re.findall(r"b[a-z0-9]{4,}", e)
      for id in match:
        url2= "".join(["https://gallica.bnf.fr/ark:/12148/", id, '.texteBrut'])
        request = urllib.request.Request(url2)
        try:
          content = urllib.request.urlopen(request)
          parse = BeautifulSoup(content, 'html.parser')
          text1 = parse.find_all('body')
          titre = parse.find_all('meta', {'name' : 'title'})
          titre = re.search(r'"(.*?)"',str(titre)).group(0)
          titre = re.sub('[\"\/]', '', titre)
          titre = re.sub(' ', '_', titre)
          print(titre + ' en cours de téléchargement...')
          date = parse.find_all('meta', {'name' : 'DC.date'})
          date = re.search(r'"(.*?)"',str(date)).group(0)
          date = re.sub('"', '', date)
          filename = "".join(date+'_'+titre[:45]+'_'+id+'_texte_brut.txt')
          text2 = re.sub('<.*?>', '', str(text1))
          text5 = re.sub("\d+", "", text2)
          text6 = re.sub('Rappel?(.*?)%.','',text5, flags=re.DOTALL)
          with open(filename, 'w') as text_file:
            text_file.write(str(text6))
        except HTTPError as tp:
          print("Le contenu de ce lien n'a pas pu être téléchargé : " + e)
  print("Fin du téléchargement.")
  chemin_nouveau_dossier = ''.join(chemin_vers_le_dossier_de_travail+"/"+ nom_dossier_telechargement+"/*.txt")
  for files in glob.glob(chemin_nouveau_dossier):
    taille=os.path.getsize(files)
    if taille < 10:
      nom_suppr = os.path.basename(files)
      os.remove(files)
      print("Ce fichier n'est pas océrisé : " + nom_suppr)



#@markdown ###Indiquer le chemin absolu vers le dossier de travail :

chemin_vers_le_dossier_de_travail = '/content/drive/My Drive/tutoriel_extraction_cartographie-main/'#@param {type:"string"}
#@markdown Exemple: '/content/drive/My Drive/corpus/


#@markdown ###Indiquer le chemin absolu vers le fichier excel (au format .xlsx)
chemin_vers_le_fichier_excel = '/content/drive/My Drive/tutoriel_extraction_cartographie-main/Corpus de guides paris.xlsx' #@param {type:"string"}
#@markdown Exemple: '/content/drive/My Drive/corpus/guides.xlsx

#@markdown ###Indiquer le nom de la feuille du fichier excel (il s'agit des onglets en bas du document)
nom_de_la_feuille = 'Guides Paris' #@param {type:"string"}
#@markdown Exemple : Feuille 1

#@markdown ###Indiquer la lettre de la colonne avec les liens :
lettre_de_la_colonne = 'D' #@param {type:"string"}
#@markdown Exemple : D

#@markdown ###Indiquer le nom du dossier qui sera créé dans lequel les fichiers txt vont être téléchargés:
nom_dossier_telechargement = 'fichiers textes guides gallica' #@param {type:"string"}


os.chdir(chemin_vers_le_dossier_de_travail)
if not os.path.exists(nom_dossier_telechargement):
      os.makedirs(nom_dossier_telechargement)
os.chdir(nom_dossier_telechargement)

txt_gallica_xls(chemin_vers_le_fichier_excel)